In [1]:
import pandas as pd
import numpy as np
from matplotlib import pyplot as plt
from datetime import datetime as dt
from datetime import date
from pandas.tseries.holiday import USFederalHolidayCalendar
from pandas.tseries.offsets import CustomBusinessDay
from dateutil.relativedelta import relativedelta
import os
import itertools as it
import _pickle as cPickle
import gc
import sys

pd.set_option('display.max_columns', 100)
pd.set_option('display.max_rows', 100)

old_err_state = np.seterr(divide='raise')
ignored_states = np.seterr(**old_err_state)

In [2]:
def load_cpickle_gc(filename):
    file_Name = filename
    output = open(file_Name, 'rb')

    gc.disable()

    mydict = cPickle.load(output)

    gc.enable()
    output.close()
    return mydict

In [3]:
_lambda_ = load_cpickle_gc('/Users/yiliwang/Documents/Berkeley MFE/AFP/lambda estimation/2016-12-05_LAMBDAS')

In [6]:
def my_reshape(df, maxsize = 20):
    if df.shape == (0,0):
        return default
    return df.reindex(np.arange(maxsize)).fillna(0).T.reindex(np.arange(maxsize)).fillna(0).T

In [8]:
# set up default dataframe with max shape
maxsize = 20
default = pd.DataFrame(np.zeros((20,20)), index = np.arange(maxsize), columns = np.arange(maxsize))

In [26]:
def nb_choice(level):
    return ['T_{}_{}_p'.format(level[0],level[1]),'T_{}_{}_n'.format(level[0],level[1]),
            'N_{}_{}_p'.format(level[0],level[1]),'N_{}_{}_n'.format(level[0],level[1])]

In [109]:
def weekly_date_slicer(DATA):
    res = []
    for i in range(int(len(DATA.keys())/5)):
        res.append(list(DATA.keys())[5*i:i*5+5])
    return res

In [263]:
def _oneOverT_converter_(DATA, convert_to = "daily"):
    # organize data to daily level
    data = {d: {l: {nb: default for nb in nb_choice(l)} for l in ['B1','B2','A1','A2']}\
         for d in DATA.keys()}

    for date in DATA.keys():
        print("working on ", date)
        for time in DATA[date].keys():  
            for level in ['B1','B2','A1','A2']:
                for nb in nb_choice(level):
                    try:
                        data[date][level][nb] = data[date][level][nb] + \
                                    my_reshape(DATA[date][time][level][nb])
                    except:
                        print("']['".join([date, time, level, nb]))
                        sys.exit()
    if convert_to == "daily":
        return data
    week_dates = weekly_date_slicer(data)
    if convert_to == "weekly":
        start_week = [week_dates[i][0] for i in range(len(week_dates))]
        data_weekly = {d: {l: {nb: default for nb in nb_choice(l)} for l in ['B1','B2','A1','A2']}\
                 for d in start_week}
        for level in ['B1','B2','A1','A2']:
            for nb in nb_choice(level):
                for i in range(len(week_dates)):
                    data_weekly[week_dates[i][0]][level][nb] = data[week_dates[i][0]][level][nb]+data[week_dates[i][1]][level][nb]+data[week_dates[i][2]][level][nb]+\
                    data[week_dates[i][3]][level][nb]+data[week_dates[i][4]][level][nb]
        return data_weekly
    if convert_to == 'monthly':
        dates = pd.to_datetime(list(data.keys())).to_frame()
        dic_by_month = dates.groupby([pd.Grouper(freq = 'M')]).groups
        dic_by_month={k.month: v for k, v in dic_by_month.items()}

        data_monthly = {d: {l: {nb: default for nb in nb_choice(l)} for l in ['B1','B2','A1','A2']}\
                 for d in dic_by_month.keys()}

        for level in ['B1','B2','A1','A2']:
            for nb in nb_choice(level):
                for i in range(1, 12):
                    for j in range(len(dic_by_month[i])):
                        data_monthly[i][level][nb] = data_monthly[i][level][nb]+ data[str(dic_by_month[i][j])[:10]][level][nb]
        return data_monthly

In [265]:
# test to get weekly 1/t dictionary
OneOverT_weekly = _oneOverT_converter_(_lambda_['TWTR'], convert_to = "weekly")

working on  2016-01-04
working on  2016-01-05
working on  2016-01-06
working on  2016-01-07
working on  2016-01-08
working on  2016-01-11
working on  2016-01-12
working on  2016-01-13
working on  2016-01-14
working on  2016-01-15
working on  2016-01-19
working on  2016-01-20
working on  2016-01-21
working on  2016-01-22
working on  2016-01-25
working on  2016-01-26
working on  2016-01-27
working on  2016-01-28
working on  2016-01-29
working on  2016-02-01
working on  2016-02-02
working on  2016-02-03
working on  2016-02-04
working on  2016-02-05
working on  2016-02-08
working on  2016-02-09
working on  2016-02-10
working on  2016-02-11
working on  2016-02-12
working on  2016-02-16
working on  2016-02-17
working on  2016-02-18
working on  2016-02-19
working on  2016-02-22
working on  2016-02-23
working on  2016-02-24
working on  2016-02-25
working on  2016-02-26
working on  2016-02-29
working on  2016-03-01
working on  2016-03-02
working on  2016-03-03
working on  2016-03-04
working on 

In [270]:
OneOverT_weekly.keys()

dict_keys(['2016-01-04', '2016-01-11', '2016-01-19', '2016-01-26', '2016-02-02', '2016-02-09', '2016-02-17', '2016-02-24', '2016-03-02', '2016-03-09', '2016-03-16', '2016-03-23', '2016-03-31', '2016-04-07', '2016-04-14', '2016-04-21', '2016-04-28', '2016-05-05', '2016-05-12', '2016-05-19', '2016-05-26', '2016-06-03', '2016-06-10', '2016-06-17', '2016-06-24', '2016-07-01', '2016-07-11', '2016-07-18', '2016-07-25', '2016-08-01', '2016-08-08', '2016-08-15', '2016-08-22', '2016-08-29', '2016-09-06', '2016-09-13', '2016-09-20', '2016-09-27', '2016-10-04', '2016-10-12', '2016-10-19', '2016-10-26', '2016-11-02', '2016-11-09', '2016-11-17', '2016-11-25'])

In [285]:
def _lambda_calculator_(DIC):
    res_dic = {d: {l: {nb: default for nb in ['lambda_positive','lambda_negative']} for l in ['B1','B2','A1','A2']}\
         for d in DIC.keys()}
    for date in list(DIC.keys()):
        for level in ['B1','B2','A1','A2']:
            T_P_label = 'T_'+level[0]+'_'+level[1]+'_p'
            N_P_label = 'N_'+level[0]+'_'+level[1]+'_p'
            T_N_label = 'T_'+level[0]+'_'+level[1]+'_n'
            N_N_label = 'N_'+level[0]+'_'+level[1]+'_n'
            res_dic[date][level]['lambda_positive'] = (DIC[date][level][T_P_label]/DIC[date][level][N_P_label]).fillna(0)
            res_dic[date][level]['lambda_negative'] = (DIC[date][level][T_N_label]/DIC[date][level][N_N_label]).fillna(0)
    return res_dic

In [287]:
# test to generate weekly lambda
lambda_weekly = _lambda_calculator_(OneOverT_weekly)

In [295]:
# sample output
lambda_weekly['2016-01-11']['A1']['lambda_positive']

,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19
0,6261.301854,6361.295033,17045.111746,16619.721109,20784.240547,13219.286571,2.198929e+04,23245.569113,1.245848e+04,8769.724560,28065.637226,17262.996650,6340.301065,39397.843388,14037.451291,68523.675044,855.610988,3813.099094,7820.024287,955.723262
1,5739.403865,10725.081686,41751.470870,12170.756261,14320.296029,23081.918721,7.671798e+03,34781.040337,6.354835e+03,25130.915784,14055.783485,8366.540535,24665.063482,6704.108946,18617.516543,26824.328646,5456.600286,74370.743725,6683.160221,6521.781680
2,18671.562518,13772.380814,41201.897856,14085.462966,53479.586666,25513.733278,1.203338e+04,55262.874843,2.775861e+04,23295.313419,10923.926615,23067.586583,50760.535684,17043.502405,14485.390307,31546.647950,846.599438,23087.837021,11854.416323,3493.701146
3,4163.836044,17165.082120,34206.660859,30709.119697,26039.242227,24154.102741,7.989336e+03,11357.135091,2.064883e+04,17082.154942,71760.886745,89567.375179,44937.230652,6580.075722,4534.155702,26916.653518,2805.611287,49868.791355,35327.673030,101331.946612
4,6281.689448,13550.869215,9995.725349,25708.401084,29286.712159,34533.616190,4.977443e+04,88963.565891,1.929021e+04,47689.618668,11623.134758,153984.507857,20219.523633,3878.731282,12976.555651,25828.405969,8663.614045,441086.916784,20463.559937,5336.557895
5,15188.559825,39577.296365,8007.107951,22019.267189,39609.885630,42903.165745,5.706097e+04,29241.614872,5.526053e+04,19424.528170,5087.011480,74376.278549,118662.728085,21610.364134,111538.774017,211033.510505,18104.327554,29968.104278,51174.082830,19539.035281
6,74930.558691,33220.215095,44301.882521,24365.212105,29032.198722,31069.360918,5.475462e+04,12053.362322,1.667078e+04,25808.049088,11167.066660,20532.139320,83336.222411,8508.977267,25304.490165,3409.337070,27383.487731,91148.249014,204.248164,279.543565
7,71008.578772,38292.080478,41243.735838,31698.035398,20398.960339,45927.621186,4.488243e+04,29887.427637,2.272392e+04,72360.254483,9496.891694,61579.118078,80615.041575,12616.953531,76365.599821,234397.839366,12786.620576,10920.963415,28156.714248,695.831134
8,53890.195842,21368.273803,88728.616584,24727.544923,201427.212982,66945.198751,2.363274e+05,46845.316485,2.010863e+05,13585.225658,9299.181308,2757.710060,11595.471819,353312.319960,158992.463241,49345.257903,1556.447064,15750.862543,11618.044703,16.696454
9,45071.956877,12269.363870,54987.477549,37275.221577,116500.407170,22318.342625,6.371562e+04,14526.536961,3.857071e+03,101569.309414,21562.072728,8665.507240,0.000000,82501.447617,230494.119030,5776.627817,3767.385004,1035.619706,28949.047800,150019.260478


In [299]:
# save output 
import pickle

def dump_pickle(file, filename):
    file_Name = filename
    fileObject = open(file_Name, 'wb')
    cPickle.dump(file, fileObject)
    fileObject.close()

In [300]:
dump_pickle(lambda_weekly, "lambda_weekly")

In [154]:
"""
week_dates = weekly_date_slicer(data)
start_week = [week_dates[i][0] for i in range(len(week_dates))]
data_weekly = {d: {l: {nb: default for nb in nb_choice(l)} for l in ['B1','B2','A1','A2']}\
         for d in start_week}
for level in ['B1','B2','A1','A2']:
    for nb in nb_choice(level):
        for i in range(len(week_dates)):
            data_weekly[week_dates[i][0]][level][nb] = data[week_dates[i][0]][level][nb]+data[week_dates[i][1]][level][nb]+data[week_dates[i][2]][level][nb]+\
            data[week_dates[i][3]][level][nb]+data[week_dates[i][4]][level][nb]
            """

In [255]:
"""
dates = pd.to_datetime(list(data.keys())).to_frame()
dic_by_month = dates.groupby([pd.Grouper(freq = 'M')]).groups
dic_by_month={k.month: v for k, v in dic_by_month.items()}

data_monthly = {d: {l: {nb: default for nb in nb_choice(l)} for l in ['B1','B2','A1','A2']}\
         for d in dic_by_month.keys()}

for level in ['B1','B2','A1','A2']:
    for nb in nb_choice(level):
        for i in range(1, 12):
            for j in range(len(dic_by_month[i])):
                data_monthly[i][level][nb] = data_monthly[i][level][nb]+ data[str(dic_by_month[i][j])[:10]][level][nb]
                """

In [266]:
"""
# organize data to daily level
data = {d: {l: {nb: default for nb in nb_choice(l)} for l in ['B1','B2','A1','A2']}\
     for d in _lambda_['TWTR'].keys()}

for date in _lambda_['TWTR'].keys():
    print("working on ", date)
    for time in _lambda_['TWTR'][date].keys():  
        for level in ['B1','B2','A1','A2']:
            for nb in nb_choice(level):
                try:
                    data[date][level][nb] = data[date][level][nb] + \
                                my_reshape(_lambda_['TWTR'][date][time][level][nb])
                except:
                    print("']['".join([date, time, level, nb]))
                    sys.exit()
                    """

'\n# organize data to daily level\ndata = {d: {l: {nb: default for nb in nb_choice(l)} for l in [\'B1\',\'B2\',\'A1\',\'A2\']}     for d in _lambda_[\'TWTR\'].keys()}\n\nfor date in _lambda_[\'TWTR\'].keys():\n    print("working on ", date)\n    for time in _lambda_[\'TWTR\'][date].keys():  \n        for level in [\'B1\',\'B2\',\'A1\',\'A2\']:\n            for nb in nb_choice(level):\n                try:\n                    data[date][level][nb] = data[date][level][nb] +                                 my_reshape(_lambda_[\'TWTR\'][date][time][level][nb])\n                except:\n                    print("\'][\'".join([date, time, level, nb]))\n                    sys.exit()\n                    '

In [ ]:
from statsmodels.tsa.stattools import adfuller

result = adfuller(crime_num_df['INCIDENT_NUMBER'], autolag='AIC')
print(f'ADF Statistic: {result[0]}')
print(f'p-value: {result[1]}')
for key, value in result[4].items():
    print('Critial Values:')
    print(f'   {key}, {value}')